## Libraries setup

In [1]:
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install textflint
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.2 MB/s eta 0:00:00
   

In [2]:
from google.colab import drive
import sys

drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/UNIVERSIDAD/TFG")

Mounted at /content/drive


## Load sample dataset from huggingface

I this example we'll use the Stanford Sentiment Treebank dataset (sst2), that consists of single sentences extracted from movies reviews. Evidently, this datasets can be used for Sentiment Analysis 

In [3]:
from datasets import load_dataset

glue_sst2 = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

If we take a look at the distinct labels in each partition:

In [4]:
print(f'Unique labels in train: {set(glue_sst2["train"]["label"])}')
print(f'Unique labels in validation: {set(glue_sst2["validation"]["label"])}')
print(f'Unique labels in test: {set(glue_sst2["test"]["label"])}')

Unique labels in train: {0, 1}
Unique labels in validation: {0, 1}
Unique labels in test: {-1}


Turns out the labels for the test partition are not publicly available, so we can only use train and validation

## Custom class: PerturbedDataset

We'll use a custom class for convience and easy access to methods

In [5]:
import Transformations as Trf2

sst2_train = Trf2.PerturbedDataset(glue_sst2["train"])
sst2_val = Trf2.PerturbedDataset(glue_sst2["validation"])

sst2_train

TextFlint: Downloading http://textflint.oss-cn-beijing.aliyuncs.com/download/NLTK_DATA/wordnet/wordnet.zip.
100%|██████████| 10.8M/10.8M [00:10<00:00, 1.08MB/s]
TextFlint: Unzipping file /root/.cache/textflint/tmpvfogt23s to /root/.cache/textflint/NLTK_DATA/wordnet.
TextFlint: Successfully saved NLTK_DATA/wordnet/wordnet.zip to cache.


Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

## Choosing a perturbation: OCR (Optical Character Recongnition)

In this example we'll use a OCR (Optical Character Recognition) perturbation, which consists of replacing characters with similar looking ones, the ones you could mistake with when looking at them. We'll perturb 10% of the characters in the sentence

In [6]:
ocr_pert = Trf2.CharacterPerturbation(perturbation_type = "Ocr", proportion_characters = 0.1)

## Applying the perturbation to the dataset

In [7]:
sst2_ocr_pert_train, perc_altered_train = sst2_train.perturb(perturbation = ocr_pert, x_fields = "sentence")
sst2_ocr_pert_val, perc_altered_val = sst2_val.perturb(perturbation = ocr_pert, x_fields = "sentence")

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

TextFlint: Downloading http://textflint.oss-cn-beijing.aliyuncs.com/download/SPACY_MODEL/model.zip.

  0%|          | 0.00/764M [00:00<?, ?B/s]
  0%|          | 14.3k/764M [00:00<3:51:37, 55.0kB/s]
  0%|          | 44.0k/764M [00:00<2:04:08, 103kB/s] 
  0%|          | 89.1k/764M [00:00<1:20:47, 158kB/s]
  0%|          | 152k/764M [00:00<59:06, 215kB/s]   
  0%|          | 315k/764M [00:01<31:42, 401kB/s]
  0%|          | 539k/764M [00:01<22:12, 573kB/s]
  0%|          | 772k/764M [00:01<18:54, 672kB/s]
  0%|          | 1.02M/764M [00:01<16:39, 763kB/s]
  0%|          | 1.27M/764M [00:02<14:12, 894kB/s]
  0%|          | 1.54M/764M [00:02<12:39, 1.00MB/s]
  0%|          | 1.83M/764M [00:02<12:03, 1.05MB/s]
  0%|          | 2.12M/764M [00:02<11:46, 1.08MB/s]
  0%|          | 2.43M/764M [00:03<11:31, 1.10MB/s]
  0%|          | 2.75M/764M [00:03<11:18, 1.12MB/s]
  0%|          | 3.09M/764M [00:03<10:59, 1.15MB/s]
  0%|          | 3.44M/764M [00:03<10:02, 1.26MB/s]
  0%|          | 3.81M/764

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Here's an example of the perturbation

In [8]:
print(f'|||{sst2_train[0]["sentence"]}||| vs |||{sst2_ocr_pert_train[0]["sentence"]}|||')

|||hide new secretions from the parental units ||| vs |||hide new 8ecretions from the parenta1 unit8|||


Now the % of altered samples is returned so we can better decide if the perturbation really altered the dataset enough

## Load the model and prepare the datasets

The `to_processed_nlp_dataset` method of the `PerturbedDataset` class inmediately prepares the dataset to be given to a model to train

In [9]:
model_name = "bert-base-uncased"

to_train = sst2_train.to_processed_nlp_dataset(
    checkpoint = model_name,
    x_fields = "sentence",
    y_fields = "label"
)

to_val = sst2_val.to_processed_nlp_dataset(
    checkpoint = model_name,
    x_fields = "sentence",
    y_fields = "label"
)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Likewise, the `load_classifier_model` loads the model from a checkpoint and attaches a classifier head at the end

In [41]:
model = Trf2.load_classifier_model(checkpoint = model_name, num_labels = 2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Aditional configuration (dynamic learning rate)

In [42]:
from tensorflow import keras

n_epochs = 3

adam_optimizer = Trf2.configure_dynamic_lr(
    scheduler_fn = keras.optimizers.schedules.PolynomialDecay,
    num_train_steps = len(to_train) * n_epochs
)

### Compilation and training

This model is going to use only the train paritition that is not perturbed

In [43]:
model.compile(
    optimizer = adam_optimizer,
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ["accuracy"]
)

history = model.fit(
    to_train,
    epochs = n_epochs
)

Epoch 1/3
2105/2105 [==============================] - 624s 272ms/step - loss: 0.2024 - accuracy: 0.9229
Epoch 2/3
2105/2105 [==============================] - 556s 264ms/step - loss: 0.0943 - accuracy: 0.9682
Epoch 3/3
2105/2105 [==============================] - 555s 264ms/step - loss: 0.0459 - accuracy: 0.9841


In [44]:
history.history

{'loss': [0.2024335116147995, 0.09426113218069077, 0.04592100530862808],
 'accuracy': [0.9228644967079163, 0.9682252407073975, 0.9840531945228577]}

### Predictions and evaluation

Since we used the training non-perturbed partitoin, let's first evaluate the dataset in the validation, non-perturbed partition:

In [45]:
import numpy as np

preds_logits = model.predict(to_val).logits
preds = np.argmax(preds_logits, axis = 1)

28/28 [==============================] - 8s 113ms/step


In [46]:
from sklearn.metrics import classification_report

evaluation = classification_report(y_true = sst2_val["label"], y_pred = preds, target_names = ["Positive", "Negative"])
print(evaluation)

              precision    recall  f1-score   support

    Positive       0.49      0.48      0.49       428
    Negative       0.51      0.52      0.51       444

    accuracy                           0.50       872
   macro avg       0.50      0.50      0.50       872
weighted avg       0.50      0.50      0.50       872



Let's now perturb the validation dataset and use it to predict in the model, to see how the predictions change

In [47]:
to_val_pert = Trf2.PerturbedDataset(dataset = sst2_ocr_pert_val).to_processed_nlp_dataset(
    checkpoint = model_name,
    x_fields = "sentence",
    y_fields = "label"
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [48]:
preds_pert_logits = model.predict(to_val_pert).logits
preds_pert = np.argmax(preds_pert_logits, axis = 1)

28/28 [==============================] - 5s 171ms/step


In [49]:
evaluation_pert = classification_report(y_true = sst2_val["label"], y_pred = preds_pert, target_names = ["Positive", "Negative"])
print(evaluation_pert)

              precision    recall  f1-score   support

    Positive       0.49      0.82      0.61       428
    Negative       0.50      0.17      0.25       444

    accuracy                           0.49       872
   macro avg       0.49      0.50      0.43       872
weighted avg       0.49      0.49      0.43       872



### Comparison in predictions between perturbed and non-perturbed inputs

In [50]:
from sklearn.metrics import cohen_kappa_score

agreement = cohen_kappa_score(preds, preds_pert, labels = [0, 1])

In [51]:
agreement

-0.008500159699970089

In [52]:
discrepancies = np.where(preds != preds_pert)[0]

### Some examples of discrepancies

In [53]:
for i in range(min(5, len(discrepancies))):
  index = int(discrepancies[i])
  prediction, pert_prediction = preds[index], preds_pert[index]
  if prediction == 0:
    prediction = "(negative)"
  else:
    prediction = "(positive)"

  if pert_prediction == 0:
    pert_prediction = "(negative)"
  else:
    pert_prediction = "(positive)"

  original_input = sst2_val['sentence'][index]
  perturbed_input = sst2_ocr_pert_val['sentence'][index]

  cad = "-" * 150 + "\n"
  cad += f"Original input:\n\t'{original_input}'\nPrediction: {prediction}\n\n"
  cad += f"Perturbed input:\n\t'{perturbed_input}'\nPrediction: {pert_prediction}\n"

  print(cad)

------------------------------------------------------------------------------------------------------------------------------------------------------
Original input:
	'unflinchingly bleak and desperate '
Prediction: (positive)

Perturbed input:
	'unf1inchingly blear and de8perate'
Prediction: (negative)

------------------------------------------------------------------------------------------------------------------------------------------------------
Original input:
	'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . '
Prediction: (negative)

Perturbed input:
	'al1ows os tu hupe that nolan is puised t0 embarr a major career a8 a commekcial yet inventive filmmakek.'
Prediction: (positive)

------------------------------------------------------------------------------------------------------------------------------------------------------
Original input:
	'the acting , costumes , music , cinematography and sound are all astoundin

## The other way around: training a model with perturbed inputs

Let's now do the opposite: use the perturbed train partition to train the model. We'll then evaluate this new model in the same fashion as before

In [32]:
model_pert = Trf2.load_classifier_model(checkpoint = model_name, num_labels = 2)

n_epochs = 3

adam_optimizer = Trf2.configure_dynamic_lr(
    scheduler_fn = keras.optimizers.schedules.PolynomialDecay,
    num_train_steps = len(to_train) * n_epochs
)

to_train_pert = Trf2.PerturbedDataset(dataset = sst2_ocr_pert_train).to_processed_nlp_dataset(
    checkpoint = model_name,
    x_fields = "sentence",
    y_fields = "label"
)

model_pert.compile(
    optimizer = adam_optimizer,
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ["accuracy"]
)

history = model_pert.fit(
    to_train_pert,
    epochs = n_epochs
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 1/3
2105/2105 [==============================] - 942s 425ms/step - loss: 0.3943 - accuracy: 0.8163
Epoch 2/3
2105/2105 [==============================] - 861s 409ms/step - loss: 0.1696 - accuracy: 0.9344
Epoch 3/3
2105/2105 [==============================] - 863s 410ms/step - loss: 0.0630 - accuracy: 0.9773


In [33]:
new_preds_logits = model_pert.predict(to_val).logits
new_preds = np.argmax(new_preds_logits, axis = 1)

new_preds_pert_logits = model_pert.predict(to_val_pert).logits
new_preds_pert = np.argmax(preds_pert_logits, axis = 1)

28/28 [==============================] - 5s 168ms/step


In [34]:
new_evaluation = classification_report(y_true = sst2_val["label"], y_pred = new_preds, target_names = ["Positive", "Negative"])
print(new_evaluation)

              precision    recall  f1-score   support

    Positive       0.49      0.46      0.48       428
    Negative       0.51      0.54      0.53       444

    accuracy                           0.50       872
   macro avg       0.50      0.50      0.50       872
weighted avg       0.50      0.50      0.50       872



In [35]:
new_evaluation_pert = classification_report(y_true = sst2_val["label"], y_pred = new_preds_pert, target_names = ["Positive", "Negative"])
print(new_evaluation)

              precision    recall  f1-score   support

    Positive       0.49      0.46      0.48       428
    Negative       0.51      0.54      0.53       444

    accuracy                           0.50       872
   macro avg       0.50      0.50      0.50       872
weighted avg       0.50      0.50      0.50       872



In [36]:
print(f"Agreement: {cohen_kappa_score(new_preds, new_preds_pert, labels = [0, 1])}")

Agreement: -0.0016445237359592468


In [37]:
new_discrepancies = np.where(new_preds != new_preds_pert)[0]

In [54]:
for i in range(min(5, len(new_discrepancies))):
  index = int(new_discrepancies[i])
  prediction, pert_prediction = new_preds[index], new_preds_pert[index]
  if prediction == 0:
    prediction = "(negative)"
  else:
    prediction = "(positive)"

  if pert_prediction == 0:
    pert_prediction = "(negative)"
  else:
    pert_prediction = "(positive)"

  original_input = sst2_val['sentence'][index]
  perturbed_input = sst2_ocr_pert_val['sentence'][index]

  cad = "-" * 150 + "\n"
  cad += f"Original input:\n\t'{original_input}'\nPrediction: {prediction}\n\n"
  cad += f"Perturbed input:\n\t'{perturbed_input}'\nPrediction: {pert_prediction}\n"

  print(cad)

------------------------------------------------------------------------------------------------------------------------------------------------------
Original input:
	'unflinchingly bleak and desperate '
Prediction: (positive)

Perturbed input:
	'unf1inchingly blear and de8perate'
Prediction: (negative)

------------------------------------------------------------------------------------------------------------------------------------------------------
Original input:
	'it 's slow -- very , very slow . '
Prediction: (negative)

Perturbed input:
	'it's s1ow -- very, veky 8low.'
Prediction: (positive)

------------------------------------------------------------------------------------------------------------------------------------------------------
Original input:
	'or doing last year 's taxes with your ex-wife . '
Prediction: (positive)

Perturbed input:
	'or doing 1ast yeak'8 taxe8 with your ex-wife.'
Prediction: (negative)

----------------------------------------------------------